In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import math
from scipy.stats import norm
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#CONFIDENCE INTERVAL

def confidence_interval(mu, sigma, conf_level, time):
    
    Z_value = norm.ppf(conf_level+(1-conf_level)/2)
    lower_bound = (mu-sigma**2/2) - Z_value*sigma/np.sqrt(time)
    upper_bound = (mu-sigma**2/2) + Z_value*sigma/np.sqrt(time)

    return(round(lower_bound, 4), round(upper_bound, 4))

In [3]:
#VOLITILITY MEASUREMENT

def volitility_measurement(x, observed_type):

    #the u_i
    u_i = np.zeros(len(x)-1)

    for i in range(1, len(x)):
        u_i[i-1] = np.log(x[i]/x[i-1])
    
    #calculate the mean of u_i 
    mean_ui = np.nanmean(u_i)

    #calculate s
    s = 0

    for i in range(0, len(u_i)):
        s += (u_i[i] - mean_ui)**2

    s_final = np.sqrt(1/(len(u_i)-1)*s)

    #Figure out which t value to use
    if observed_type == "daily":
        t = 252
    elif observed_type == "monthly":
        t = 12
    elif observed_type == "weekly":
        t = 52
    else:
        return "Not a valid type, must be either daily, monthly, or weekly."

    #CALCULATE sigma
    sigma = s_final*np.sqrt(t)

    #OUTPUTS
    print("s_value = " + str(s_final))
    print("estimated annual volitility = " + str(sigma))
    print("estimated return = " + str(mean_ui*t+sigma**2/2))

    return

In [4]:
# Z - CALCULATOR

def z_calulator(prob):
    
    print("Z = " + str(norm.ppf(prob)))
    print("Z/2 = " + str(norm.ppf(prob+(1-prob)/2)))

    return

In [5]:
# ARITHMETIC BROWNIAN MOTION PROB CALCULATOR

def arithmetic_brownian_motion(drift, diffusion, boundaries, T, interval_type):
    if interval_type != "left" and interval_type != "right" and interval_type != "both":
        return "Enter a valid interval type"

    #CALCULATE THE Z VALUE 
    Z_1 = (boundaries[0] - drift*T)/(diffusion*np.sqrt(T))
    if interval_type == "both":
        Z_2 = (boundaries[1] - drift*T)/(diffusion*np.sqrt(T))

    #CALCULATE PROBABILITIES
    if interval_type == "left":
        answer = norm.cdf(Z_1)
    elif interval_type == "right":
        answer = 1 - norm.cdf(Z_1)
    elif interval_type == "both":
        answer = abs(norm.cdf(Z_2) - norm.cdf(Z_1))

    return answer
    

In [6]:
# GEOMETRIC BROWNIAN MOTION PROB CALCULATOR

def geometric_brownian_motion(drift, diffusion, S_0, boundaries, T, interval_type):
    if interval_type != "left" and interval_type != "right" and interval_type != "both":
        return "Enter a valid interval type"

    #CALCULATE THE Z VALUE 
    Z_1 = (np.log(boundaries[0]) - (np.log(S_0) + (drift-diffusion**2/2)*T))/(diffusion*np.sqrt(T))
    if interval_type == "both":
        Z_2 = (np.log(boundaries[1]) - (np.log(S_0) + (drift-diffusion**2/2)*T))/(diffusion*np.sqrt(T))

    #CALCULATE PROBABILITIES
    if interval_type == "left":
        answer = norm.cdf(Z_1)
    elif interval_type == "right":
        answer = 1 - norm.cdf(Z_1)
    elif interval_type == "both":
        answer = abs(norm.cdf(Z_2) - norm.cdf(Z_1))

    return answer

In [7]:
#REGULAR BLACK SCHOLES

def regular_bs(S, K, r,delta, sigma, T):

    d_1 = (np.log(S/K) + (r-delta+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)

    c = S*np.exp(-delta*T)*norm.cdf(d_1) - K*np.exp(-r*T)*norm.cdf(d_2)
    p = K*np.exp(-r*T)*norm.cdf(-d_2) - S*np.exp(-delta*T)*norm.cdf(-d_1)

    print("Call Price = " + str(c))
    print("Put Price = " + str(p))

    return

In [8]:
#FUTURES BLACK SCHOLES

def futures_bs(S, K, r, sigma, T):

    d_1 = (np.log(S/K) + (sigma**2/2)*T)/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)

    c = S*np.exp(-r*T)*norm.cdf(d_1) - K*np.exp(-r*T)*norm.cdf(d_2)
    p = K*np.exp(-r*T)*norm.cdf(-d_2) - S*np.exp(-r*T)*norm.cdf(-d_1)

    print("Call Price = " + str(c))
    print("Put Price = " + str(p))

    return

In [9]:
#CURRENCY BLACK SCHOLES


def currency_bs(S, K, r, r_foreign, sigma, T):

    d_1 = (np.log(S/K) + (r - r_foreign + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)

    c = S*np.exp(-r_foreign*T)*norm.cdf(d_1) - K*np.exp(-r*T)*norm.cdf(d_2)
    p = K*np.exp(-r*T)*norm.cdf(-d_2) - S*np.exp(-r_foreign*T)*norm.cdf(-d_1)

    print("Call Price = " + str(c))
    print("Put Price = " + str(p))

    return

In [10]:
#DISCRETE DIVIDENDS BLACK SCHOLES


def discrete_dividends_bs(S, K, r, dividend_payments, dividend_timing, sigma, T):

    PV_div = 0
    for i in range(0, len(dividend_payments)):
        PV_div += dividend_payments[i]*np.exp(-r*dividend_timing[i])

    S_0 = S - PV_div

    d_1 = (np.log(S_0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)

    c = S_0*norm.cdf(d_1) - K*np.exp(-r*T)*norm.cdf(d_2)
    p = K*np.exp(-r*T)*norm.cdf(-d_2) - S_0**norm.cdf(-d_1)

    print("Call Price = " + str(c))
    print("Put Price = " + str(p))

    return

In [42]:
def gamma(S, K, r, delta, sigma, T):
    
    d_1 = (np.log(S/K) + (r-delta+sigma**2/2)*T)/(sigma*np.sqrt(T))

    gamma_answer = norm.pdf(d_1)/(S*sigma*np.sqrt(T))

    print("Gamma = " + str(gamma_answer))

    return

In [15]:
def delta(S, K, r, delta, sigma, T):
    
    d_1 = (np.log(S/K) + (r-delta+sigma**2/2)*T)/(sigma*np.sqrt(T))

    delta_call = norm.cdf(d_1)
    delta_put = delta_call - 1

    print("Delta Call = " + str(delta_call))
    print("Delta Put = " + str(delta_put))
    
    return

In [45]:
def theta(S, K, r, delta, sigma, T):
    
    d_1 = (np.log(S/K) + (r-delta+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)

    theta_call = (-S*norm.pdf(d_1)*sigma)/(2*np.sqrt(T))-r*K*np.exp(-r*T)*norm.cdf(d_2)
    theta_put = (-S*norm.pdf(d_1)*sigma)/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d_2)

    print("Theta Call (per 1 Calander Day) = " + str(theta_call/365))
    print("Theta Call (per 1 Trading Day) = " + str(theta_call/252))
    print("Theta Put (per 1 Calander Day) = " + str(theta_put/365))
    print("Theta Put (per 1 Trading Day) = " + str(theta_put/252))
    
    return

In [50]:
def vega(S, K, r, delta, sigma, T):
    
    d_1 = (np.log(S/K) + (r-delta+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)

    vega_answer = S*np.sqrt(T)*norm.pdf(d_1)

    print("Vega (per 1%) = " + str(vega_answer/100))
    
    return

In [54]:
def rho(S, K, r, delta, sigma, T):
    
    d_1 = (np.log(S/K) + (r-delta+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)

    rho_call = K*T*np.exp(-r*T)*norm.cdf(d_2)
    rho_put = -K*T*np.exp(-r*T)*norm.cdf(-d_2)

    print("Rho Call (per 1%) = " + str(rho_call/100))
    print("Rho Put (per 1%)= " + str(rho_put/100))
    
    return

In [29]:
def psi(S, K, r, delta, sigma, T):
    
    d_1 = (np.log(S/K) + (r-delta+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)

    psi_call = -T*S*np.exp(-delta*T)*norm.cdf(d_1)
    psi_put = T*S*np.exp(-delta*T)*norm.cdf(-d_1)

    print("Psi Call (per 1%) = " + str(psi_call/100))
    print("Psi Put (per 1%) = " + str(psi_put/100))
    
    return

In [11]:
'''

#1
confidence_interval(0.09, 0.30, 0.91, 8)

#2
volitility_measurement([20.53, 22.02, 21.68, 24.77, 21.70], "daily")

#3
volitility_measurement([20.24, 24.29, 22.49, 21.73, 23.42], "daily")

#4
z_calulator(0.91)

#5
arithmetic_brownian_motion(0.42, 0.33, [1.03, 2.03], 4, "both")

#6
geometric_brownian_motion(0.35, 0.24, 40, [44], 6/12, "left")

#7
geometric_brownian_motion(0.39, 0.28, 45, [42], 8/12, "right")

#8
geometric_brownian_motion(0.35, 0.24, 40, [44, 47], 6/12, "both")

#9
geometric_brownian_motion(0.22, 0.20, 86, [128], 3, "right")

#10
futures_bs(52, 87, 0.18, 0.16, 8/12)

#11
currency_bs(0.431, 0.187, 0.13, 0.10, 0.14, 4)

#12
discrete_dividends_bs(279, 180, 0.17, [75, 42, 33], [3, 7, 10], 0.19, 18)

#13
theta(33, 82, 0.285, 0, 0.2945, 14/52)

#14
psi(58, 170, 0.05, 0.05, 0.39, 32/52)

#15
delta(49, 50, 0.05, 0, 0.20, 20/52)

#16
gamma(49, 50, 0.05, 0, 0.20, 20/52)

#17
theta(49, 50, 0.05, 0, 0.20, 20/52)

#18
vega(49, 50, 0.05, 0, 0.20, 20/52)

#19
rho(49, 50, 0.05, 0, 0.20, 20/52)
'''

'\n\n#1\nconfidence_interval(0.09, 0.30, 0.91, 8)\n\n#2\nvolitility_measurement([20.53, 22.02, 21.68, 24.77, 21.70], "daily")\n\n#3\nvolitility_measurement([20.24, 24.29, 22.49, 21.73, 23.42], "daily")\n\n#4\nz_calulator(0.91)\n\n#5\narithmetic_brownian_motion(0.42, 0.33, [1.03, 2.03], 4, "both")\n\n#6\ngeometric_brownian_motion(0.35, 0.24, 40, [44], 6/12, "left")\n\n#7\ngeometric_brownian_motion(0.39, 0.28, 45, [42], 8/12, "right")\n\n#8\ngeometric_brownian_motion(0.35, 0.24, 40, [44, 47], 6/12, "both")\n\n#9\ngeometric_brownian_motion(0.22, 0.20, 86, [128], 3, "right")\n\n#10\nfutures_bs(52, 87, 0.18, 0.16, 8/12)\n\n#11\ncurrency_bs(0.431, 0.187, 0.13, 0.10, 0.14, 4)\n\n#12\ndiscrete_dividends_bs(279, 180, 0.17, [75, 42, 33], [3, 7, 10], 0.19, 18)\n\n'